# The Death of the 12th Man: How Home Advantage Is Fading in the Premier League

**Author:** Ammar Tyabji · [@AmmardoesStats](https://github.com/AmmardoesStats)  
**Blog post:** [Read the full article](https://ammartyabji.wixsite.com/mysite/post/the-death-of-the-12th-man-how-home-advantage-is-fading-in-the-premier-league)  
**Dataset:** [9,380 EPL matches, 2000–2025 (Kaggle)](https://www.kaggle.com/datasets/marcohuiii/english-premier-league-epl-match-data-2000-2025)

---

> *I analysed 9,380 Premier League matches across 25 seasons to trace the slow death of home advantage — and found that the real story is not about stadiums getting quieter. It is about away teams getting smarter.*

---

## What This Notebook Covers

| Part | Topic |
|------|-------|
| 1 | Data loading & feature engineering |
| 2 | Exploratory analysis — the home advantage story |
| 3 | The COVID natural experiment |
| 4 | Rolling team form features (no data leakage) |
| 5 | Predictive modelling with temporal train/test split |
| 6 | Statistical inference with `statsmodels` |
| 7 | Multinomial logit — modelling all three outcomes |
| 8 | Rolling cross-validation |
| 9 | Deep dives: fortresses, COVID by team, Big Six, derbies, draws |


## Setup

Install dependencies if needed, then import everything.

In [ ]:
# Uncomment to install dependencies
# !pip install pandas numpy matplotlib seaborn scipy statsmodels scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from scipy import stats

# Econometric inference
import statsmodels.api as sm
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.discrete.discrete_model import MNLogit

# Predictive modelling
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix, ConfusionMatrixDisplay,
)
from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.filterwarnings("ignore")

# ── Global plot style ──────────────────────────────────────────────────────
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "axes.titlesize": 14,
    "axes.titleweight": "bold",
    "axes.labelsize": 12,
    "axes.grid": True,
    "grid.alpha": 0.3,
    "figure.dpi": 120,
    "font.family": "sans-serif",
})

COLORS = {
    "home":   "#2ecc71",
    "draw":   "#95a5a6",
    "away":   "#e74c3c",
    "accent": "#3498db",
    "covid":  "#e74c3c",
}

BIG_SIX = ["Man United", "Liverpool", "Arsenal", "Chelsea", "Man City", "Tottenham"]

print("✓ All imports successful")


---
## Part 1 — Data Loading & Preparation

**Dataset:** Download `epl_final.csv` from [Kaggle](https://www.kaggle.com/datasets/marcohuiii/english-premier-league-epl-match-data-2000-2025) and place it in the same folder as this notebook.

We engineer the following base features:
- `HomeWin`, `Draw`, `AwayWin` — binary outcome indicators
- `Outcome` — numeric (0=Away, 1=Draw, 2=Home) for multinomial modelling
- `GoalDiff` — home goals minus away goals
- `CovidEra` — flag for 2019/20 and 2020/21 seasons (empty/limited crowds)
- `PostCovid` — flag for all seasons from 2019 onwards


In [ ]:
def load_and_prepare(filepath: str) -> pd.DataFrame:
    """
    Load EPL match data and engineer base features.

    Parameters
    ----------
    filepath : str
        Path to epl_final.csv (downloaded from Kaggle).

    Returns
    -------
    pd.DataFrame
        Match-level DataFrame sorted by date with engineered features.
    """
    df = pd.read_csv(filepath)
    df["MatchDate"]   = pd.to_datetime(df["MatchDate"])
    df = df.sort_values("MatchDate").reset_index(drop=True)

    # Season identifier (start year)
    df["SeasonStart"] = df["Season"].str[:4].astype(int)

    # Match outcome indicators
    df["HomeWin"] = (df["FullTimeResult"] == "H").astype(int)
    df["Draw"]    = (df["FullTimeResult"] == "D").astype(int)
    df["AwayWin"] = (df["FullTimeResult"] == "A").astype(int)

    # Numeric outcome for multinomial modelling: 0=Away, 1=Draw, 2=Home
    df["Outcome"] = df["FullTimeResult"].map({"A": 0, "D": 1, "H": 2})

    # Goal difference (positive = home team ahead)
    df["GoalDiff"] = df["FullTimeHomeGoals"] - df["FullTimeAwayGoals"]

    # COVID-era flag: 2019/20 restart + 2020/21 (mostly behind closed doors)
    df["CovidEra"]  = df["SeasonStart"].isin([2019, 2020]).astype(int)
    df["PostCovid"] = (df["SeasonStart"] >= 2019).astype(int)

    return df


# ── Load the data ─────────────────────────────────────────────────────────
# UPDATE this path to wherever you saved epl_final.csv
FILEPATH = "epl_final.csv"

df = load_and_prepare(FILEPATH)

print(f"Loaded {len(df):,} matches across {df['SeasonStart'].nunique()} seasons")
print(f"Seasons: {df['SeasonStart'].min()} – {df['SeasonStart'].max()}")
print(f"Teams:   {df['HomeTeam'].nunique()} unique")
print(f"\nColumns: {list(df.columns)}")
df.head(3)


---
## Part 2 — Exploratory Analysis: The Home Advantage Story

### 2a. Overall outcome distribution
Across the full 25-season dataset, home teams win 45.8% of matches — but this aggregate hides a long structural decline.


In [ ]:
rates = df[["HomeWin", "Draw", "AwayWin"]].mean()

fig, ax = plt.subplots(figsize=(7, 5))
bars = ax.bar(
    ["Home Win", "Draw", "Away Win"],
    rates.values,
    color=[COLORS["home"], COLORS["draw"], COLORS["away"]],
    edgecolor="white", linewidth=1.2, width=0.6,
)
for bar, val in zip(bars, rates.values):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.01,
        f"{val:.1%}", ha="center", fontsize=12, fontweight="bold",
    )

ax.set_title("Match Outcome Probabilities — EPL (2000–2025)")
ax.set_ylabel("Proportion")
ax.set_ylim(0, 0.55)
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

plt.tight_layout()
plt.savefig("outputs/fig1_outcome_distribution.png", bbox_inches="tight")
plt.show()

print(f"Home: {rates['HomeWin']:.1%}  |  Draw: {rates['Draw']:.1%}  |  Away: {rates['AwayWin']:.1%}")


### 2b. Home win rate over time — the structural decline

The LOWESS smoother (a flexible, non-parametric trend line) reveals the decline is gradual and persistent, not a single shock.


In [ ]:
import os
os.makedirs("outputs", exist_ok=True)

season_hw = df.groupby("SeasonStart")["HomeWin"].mean().reset_index()

fig, ax = plt.subplots(figsize=(11, 6))
ax.plot(
    season_hw["SeasonStart"], season_hw["HomeWin"],
    marker="o", linewidth=2, markersize=5, color=COLORS["accent"],
    label="Home Win Rate",
)

smoothed = lowess(season_hw["HomeWin"], season_hw["SeasonStart"], frac=0.35)
ax.plot(smoothed[:, 0], smoothed[:, 1], "--", color="grey",
        linewidth=2, label="LOWESS Trend")

ax.axvspan(2019, 2020.5, alpha=0.15, color=COLORS["covid"],
           label="COVID Era (No/Limited Fans)")

ax.set_title("The Decline of Home Advantage (2000–2025)")
ax.set_ylabel("Home Win Rate")
ax.set_xlabel("Season Start Year")
ax.legend(loc="upper right", fontsize=10)
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

plt.tight_layout()
plt.savefig("outputs/fig2_home_advantage_trend.png", bbox_inches="tight")
plt.show()


### 2c. Average home goal difference by season

Another lens on the same story — the margin of home dominance has halved since the early 2000s and now brushes zero.

In [ ]:
season_gd = df.groupby("SeasonStart")["GoalDiff"].agg(["mean", "sem"]).reset_index()

fig, ax = plt.subplots(figsize=(11, 6))
ax.plot(season_gd["SeasonStart"], season_gd["mean"],
        marker="o", linewidth=2, markersize=5, color=COLORS["accent"])
ax.fill_between(
    season_gd["SeasonStart"],
    season_gd["mean"] - 1.96 * season_gd["sem"],
    season_gd["mean"] + 1.96 * season_gd["sem"],
    alpha=0.2, color=COLORS["accent"], label="95% CI",
)
ax.axhline(0, color="black", linewidth=0.8)
ax.axvspan(2019, 2020.5, alpha=0.15, color=COLORS["covid"], label="COVID Era")

ax.set_title("Average Home Goal Difference by Season")
ax.set_ylabel("Goal Difference (Home − Away)")
ax.set_xlabel("Season Start Year")
ax.legend(fontsize=10)
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

plt.tight_layout()
plt.savefig("outputs/fig3_goal_diff_trend.png", bbox_inches="tight")
plt.show()


---
## Part 3 — The COVID Natural Experiment

COVID provided an accidental experiment: empty stadiums for most of two seasons. This lets us estimate how much of home advantage comes from fans specifically.

**Design:** We compare home win rates and goal differences across three periods — Pre-COVID, COVID Era (no/limited fans), and Post-COVID. This is a quasi-experimental before/during/after design. It is not a formal Difference-in-Differences (we have no cross-sectional control group), but the exogenous shock is sharp enough to be informative.

**Key finding:** Home win rate dropped from 46.5% → 41.6% during COVID (p = 0.009), but only recovered to 44.7% post-COVID — suggesting COVID *accelerated* a pre-existing trend rather than causing it.


In [ ]:
labels = [
    "Pre-COVID\n(2000–2019)",
    "COVID Era\n(2019–2021)",
    "Post-COVID\n(2021–2025)",
]
conditions = [
    df["SeasonStart"] < 2019,
    df["SeasonStart"].isin([2019, 2020]),
    df["SeasonStart"] > 2020,
]
df["Period"] = np.select(conditions, labels, default="Unknown")

period_stats = (
    df.groupby("Period", sort=False)
    .agg(
        matches     = ("HomeWin", "count"),
        home_win    = ("HomeWin", "mean"),
        home_win_se = ("HomeWin", "sem"),
        avg_gd      = ("GoalDiff", "mean"),
        gd_se       = ("GoalDiff", "sem"),
    )
    .reindex(labels)
)

# ── Statistical tests ────────────────────────────────────────────────────────
pre   = df.loc[df["Period"] == labels[0], "HomeWin"]
covid = df.loc[df["Period"] == labels[1], "HomeWin"]
post  = df.loc[df["Period"] == labels[2], "HomeWin"]

t1, p1 = stats.ttest_ind(pre, covid)
t2, p2 = stats.ttest_ind(covid, post)
print(f"Pre vs COVID:    t = {t1:.3f},  p = {p1:.4f}  {'✓ significant' if p1 < 0.05 else '✗ not significant'}")
print(f"COVID vs Post:   t = {t2:.3f},  p = {p2:.4f}  {'✓ significant' if p2 < 0.05 else '✗ not significant'}")

# ── OLS with robust SEs ──────────────────────────────────────────────────────
print("\n── OLS: HomeWin ~ CovidEra + PostCovid (HC1 robust SEs) ──")
X_ols     = sm.add_constant(df[["PostCovid", "CovidEra"]])
ols_result = sm.OLS(df["HomeWin"], X_ols).fit(cov_type="HC1")
print(ols_result.summary2().tables[1].to_string())

period_stats[["matches", "home_win", "avg_gd"]]


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5.5))
x = range(3)

# Panel A: Home Win Rate
ax = axes[0]
vals = period_stats["home_win"].values
errs = 1.96 * period_stats["home_win_se"].values
ax.bar(x, vals, yerr=errs, capsize=5, width=0.55,
       color=[COLORS["accent"], COLORS["covid"], COLORS["home"]],
       edgecolor="white", linewidth=1.2)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=10)
ax.set_ylabel("Home Win Rate")
ax.set_title("Panel A: Home Win Rate by Period")
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
for i, (v, e) in enumerate(zip(vals, errs)):
    ax.text(i, v + e + 0.01, f"{v:.1%}", ha="center", fontweight="bold")
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

# Panel B: Goal Difference
ax = axes[1]
vals_gd = period_stats["avg_gd"].values
errs_gd = 1.96 * period_stats["gd_se"].values
ax.bar(x, vals_gd, yerr=errs_gd, capsize=5, width=0.55,
       color=[COLORS["accent"], COLORS["covid"], COLORS["home"]],
       edgecolor="white", linewidth=1.2)
ax.axhline(0, color="black", linewidth=0.8)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=10)
ax.set_ylabel("Avg Goal Difference (Home − Away)")
ax.set_title("Panel B: Goal Difference by Period")
for i, (v, e) in enumerate(zip(vals_gd, errs_gd)):
    offset = e + 0.015 if v >= 0 else -(e + 0.06)
    ax.text(i, v + offset, f"{v:+.3f}", ha="center", fontweight="bold")
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

plt.suptitle(
    "The COVID Natural Experiment: Did Empty Stadiums Kill Home Advantage?",
    fontsize=14, fontweight="bold", y=1.02,
)
plt.tight_layout()
plt.savefig("outputs/fig4_covid_experiment.png", bbox_inches="tight")
plt.show()


---
## Part 4 — Feature Engineering: Rolling Team Form

We construct 5-match rolling averages for each team across five metrics: goal difference, points, goals scored, goals conceded, and red cards.

**Critical design choices to prevent data leakage:**
- `shift(1)` — each match only uses *past* results, never the current one
- `min_periods=window` — we require a full window of history before producing a feature (avoids noisy early-season estimates)
- **Differential features** (`home_form − away_form`) capture the *relative* momentum gap between the two teams, which is what drives prediction


In [ ]:
def build_form_features(df: pd.DataFrame, window: int = 5) -> pd.DataFrame:
    """
    Build rolling form features for each team with strict lag to prevent leakage.

    Parameters
    ----------
    df     : Match-level DataFrame from load_and_prepare().
    window : Rolling window length in matches (default 5).

    Returns
    -------
    pd.DataFrame with Diff_Form_* columns appended.
    """
    # Points earned per match
    df["HomePoints"] = np.select(
        [df["FullTimeResult"] == "H", df["FullTimeResult"] == "D"], [3, 1], default=0
    )
    df["AwayPoints"] = np.select(
        [df["FullTimeResult"] == "A", df["FullTimeResult"] == "D"], [3, 1], default=0
    )

    # Reshape to long (team-level) panel
    home = df[["MatchDate", "SeasonStart", "HomeTeam",
               "FullTimeHomeGoals", "FullTimeAwayGoals",
               "HomeRedCards", "HomePoints"]].copy()
    home.columns = ["MatchDate", "SeasonStart", "Team", "GF", "GA", "RedCards", "Points"]

    away = df[["MatchDate", "SeasonStart", "AwayTeam",
               "FullTimeAwayGoals", "FullTimeHomeGoals",
               "AwayRedCards", "AwayPoints"]].copy()
    away.columns = home.columns

    panel = pd.concat([home, away]).sort_values(["Team", "MatchDate"])
    panel["GD"] = panel["GF"] - panel["GA"]

    # Rolling averages — strictly lagged, no leakage
    for col in ["GD", "Points", "GF", "GA", "RedCards"]:
        panel[f"Form_{col}_{window}"] = (
            panel.groupby("Team")[col]
            .transform(lambda x: x.shift(1).rolling(window, min_periods=window).mean())
        )

    form_cols = [c for c in panel.columns if c.startswith("Form_")]

    # Merge back to match level
    home_form = panel[["MatchDate", "Team"] + form_cols].copy()
    home_form.columns = ["MatchDate", "Team"] + [f"Home{c}" for c in form_cols]

    away_form = panel[["MatchDate", "Team"] + form_cols].copy()
    away_form.columns = ["MatchDate", "Team"] + [f"Away{c}" for c in form_cols]

    df = df.merge(home_form, left_on=["MatchDate", "HomeTeam"],
                  right_on=["MatchDate", "Team"], how="left").drop(columns="Team")
    df = df.merge(away_form, left_on=["MatchDate", "AwayTeam"],
                  right_on=["MatchDate", "Team"], how="left").drop(columns="Team")

    # Differential features: home form minus away form
    for col in form_cols:
        df[f"Diff_{col}"] = df[f"Home{col}"] - df[f"Away{col}"]

    return df


df = build_form_features(df, window=5)

form_cols = [c for c in df.columns if c.startswith("Diff_")]
n_before  = len(df)
n_after   = df.dropna(subset=form_cols).shape[0]

print(f"Created {len(form_cols)} form differential features:")
for c in form_cols:
    print(f"  {c}")
print(f"\nDropped {n_before - n_after:,} matches with insufficient history (first 5 matches per team per season)")
print(f"Remaining: {n_after:,} matches")


---
## Part 5 — Predictive Modelling

### Why a temporal split and not random CV?

Random train/test splits are wrong for time-series data — they let the model train on 2024 matches and be tested on 2020 ones, which leaks future information. We train on all seasons before 2022 and test on 2022 onwards, mirroring real deployment: you only have the past.

### Three models, increasing complexity

| Model | Features | Purpose |
|-------|----------|---------|
| 1 — Season FE only | Season dummies | Baseline: controls for time trends |
| 2 — Form + Season FE | Form differentials + season dummies | **Flagship** |
| 3 — Form + Season + Team FE | Above + home/away team dummies | Robustness check |


In [ ]:
# ── Temporal split ──────────────────────────────────────────────────────────
CUTOFF_SEASON = 2022
form_features = [c for c in df.columns if c.startswith("Diff_")]
df_model = df.dropna(subset=form_features).copy()

train = df_model[df_model["SeasonStart"] < CUTOFF_SEASON]
test  = df_model[df_model["SeasonStart"] >= CUTOFF_SEASON]

print(f"Train: {len(train):,} matches ({train['SeasonStart'].min()}–{train['SeasonStart'].max()})")
print(f"Test : {len(test):,} matches  ({test['SeasonStart'].min()}–{test['SeasonStart'].max()})")

# ── Naive baselines ──────────────────────────────────────────────────────────
y_test = test["HomeWin"]
acc_always_home = y_test.mean()
majority        = train["HomeWin"].mode()[0]
acc_majority    = accuracy_score(y_test, [majority] * len(y_test))

print(f"\n── Naive Baselines ──")
print(f"  Always predict Home Win : {acc_always_home:.3f}")
print(f"  Majority class (train)  : {acc_majority:.3f}")


In [ ]:
results = {}
y_train = train["HomeWin"]

# ── Model 1: Season FE only ──────────────────────────────────────────────────
X_train_1 = pd.get_dummies(train["SeasonStart"], prefix="S", drop_first=True)
X_test_1  = pd.get_dummies(test["SeasonStart"],  prefix="S", drop_first=True)
X_test_1  = X_test_1.reindex(columns=X_train_1.columns, fill_value=0)

m1   = LogisticRegression(max_iter=1000)
m1.fit(X_train_1, y_train)
acc1 = accuracy_score(y_test, m1.predict(X_test_1))
results["Season FE Only"] = acc1
print(f"Model 1 (Season FE only):       {acc1:.3f}")

# ── Model 2: Form + Season FE — Flagship ────────────────────────────────────
season_train = pd.get_dummies(train["SeasonStart"], prefix="S", drop_first=True)
season_test  = pd.get_dummies(test["SeasonStart"],  prefix="S", drop_first=True)
season_test  = season_test.reindex(columns=season_train.columns, fill_value=0)

X_train_2 = pd.concat([train[form_features].reset_index(drop=True),
                        season_train.reset_index(drop=True)], axis=1)
X_test_2  = pd.concat([test[form_features].reset_index(drop=True),
                        season_test.reset_index(drop=True)], axis=1)

m2    = LogisticRegression(max_iter=1000)
m2.fit(X_train_2, y_train)
pred2 = m2.predict(X_test_2)
acc2  = accuracy_score(y_test, pred2)
results["Form + Season FE"] = acc2
print(f"Model 2 (Form + Season FE):     {acc2:.3f}  ← flagship  (+{acc2-acc1:.3f} over Model 1)")

# ── Model 3: Form + Season + Team FE — Robustness ────────────────────────────
ht_train = pd.get_dummies(train["HomeTeam"], prefix="HT", drop_first=True)
at_train = pd.get_dummies(train["AwayTeam"], prefix="AT", drop_first=True)
ht_test  = pd.get_dummies(test["HomeTeam"],  prefix="HT", drop_first=True).reindex(columns=ht_train.columns, fill_value=0)
at_test  = pd.get_dummies(test["AwayTeam"],  prefix="AT", drop_first=True).reindex(columns=at_train.columns, fill_value=0)

X_train_3 = pd.concat([train[form_features].reset_index(drop=True),
                        season_train.reset_index(drop=True),
                        ht_train.reset_index(drop=True),
                        at_train.reset_index(drop=True)], axis=1)
X_test_3  = pd.concat([test[form_features].reset_index(drop=True),
                        season_test.reset_index(drop=True),
                        ht_test.reset_index(drop=True),
                        at_test.reset_index(drop=True)], axis=1)

m3    = LogisticRegression(max_iter=2000, solver="saga", C=1.0)
m3.fit(X_train_3, y_train.values)
pred3 = m3.predict(X_test_3)
acc3  = accuracy_score(y_test.values, pred3)
results["Form + Season + Team FE"] = acc3
print(f"Model 3 (+ Team FE):            {acc3:.3f}  (robustness, +{acc3-acc2:.3f} over flagship)")

print(f"\n── Classification Report (Model 2 — Flagship) ──")
print(classification_report(y_test.values, pred2, target_names=["Not Home Win", "Home Win"]))


In [ ]:
# ── Model comparison chart ───────────────────────────────────────────────────
all_results = {"Always Home\n(naive)": acc_always_home, **results}

fig, ax = plt.subplots(figsize=(10, 5.5))
names  = list(all_results.keys())
accs   = list(all_results.values())
colors = [COLORS["draw"]] + [COLORS["accent"]] * len(results)

bars = ax.barh(names, accs, color=colors, edgecolor="white", linewidth=1.2, height=0.55)
for bar, val in zip(bars, accs):
    ax.text(val + 0.005, bar.get_y() + bar.get_height() / 2,
            f"{val:.1%}", va="center", fontweight="bold")

ax.set_xlim(0.35, max(accs) + 0.06)
ax.set_xlabel("Test Set Accuracy")
ax.set_title("Model Comparison: Out-of-Sample Accuracy")
ax.axvline(acc_always_home, color=COLORS["draw"], linestyle="--", alpha=0.6, label="Naive baseline")
ax.legend(fontsize=9)
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

plt.tight_layout()
plt.savefig("outputs/fig5_model_comparison.png", bbox_inches="tight")
plt.show()


---
## Part 6 — Statistical Inference with `statsmodels`

`sklearn` tells you *how accurate* a model is. `statsmodels` tells you *why* — with standard errors, z-statistics, p-values, and confidence intervals. This is where the econometrics training pays off.

We re-estimate the flagship model using `statsmodels.Logit` and then plot the marginal effect of form differential on P(Home Win).


In [ ]:
# ── Logit with proper inference ─────────────────────────────────────────────
season_dummies = pd.get_dummies(train["SeasonStart"], prefix="S", drop_first=True)
X_sm = pd.concat([train[form_features].reset_index(drop=True),
                  season_dummies.reset_index(drop=True)], axis=1)
X_sm = sm.add_constant(X_sm)
y_sm = train["HomeWin"].reset_index(drop=True)

logit_model = sm.Logit(y_sm, X_sm).fit(disp=0)

print("── Form feature coefficients ──")
summary_df = logit_model.summary2().tables[1]
form_rows  = summary_df.loc[summary_df.index.isin(form_features + ["const"])]
print(form_rows.round(4).to_string())

print(f"\nPseudo R²:     {logit_model.prsquared:.4f}")
print(f"Log-likelihood: {logit_model.llf:.1f}")
print(f"AIC:            {logit_model.aic:.1f}")
print(f"Observations:   {int(logit_model.nobs):,}")


In [ ]:
# ── Marginal effects chart ───────────────────────────────────────────────────
key_feature = "Diff_Form_GD_5"
if key_feature not in X_sm.columns:
    key_feature = [c for c in X_sm.columns if "GD" in c and "Diff" in c][0]

gd_range = np.linspace(X_sm[key_feature].quantile(0.02),
                        X_sm[key_feature].quantile(0.98), 100)

X_pred = pd.DataFrame(np.tile(X_sm.mean().values, (100, 1)), columns=X_sm.columns)
X_pred[key_feature] = gd_range
probs = logit_model.predict(X_pred)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(gd_range, probs, linewidth=2.5, color=COLORS["accent"])
ax.fill_between(gd_range, probs, alpha=0.1, color=COLORS["accent"])
ax.axhline(0.5, color="grey", linestyle="--", alpha=0.6, label="50/50")
ax.axvline(0, color="grey",   linestyle=":",  alpha=0.6)

ax.set_xlabel("Form Differential: 5-Match Goal Difference\n"
              "(← Away team in better form)          (Home team in better form →)")
ax.set_ylabel("P(Home Win)")
ax.set_title("Marginal Effect of Team Form on Home Win Probability")
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.legend(fontsize=10)

mid = 50
ax.annotate(
    f"At form parity: {probs.iloc[mid]:.1%}",
    xy=(gd_range[mid], probs.iloc[mid]),
    xytext=(gd_range[mid] + 0.5, probs.iloc[mid] + 0.06),
    arrowprops=dict(arrowstyle="->", color="grey"),
    fontsize=10, color="grey",
)
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

plt.tight_layout()
plt.savefig("outputs/fig6_marginal_effects.png", bbox_inches="tight")
plt.show()


---
## Part 7 — Multinomial Logit: All Three Outcomes

Binary classification (Home Win vs Not) throws away information by lumping draws with away wins. A multinomial logit models the full outcome distribution simultaneously.

**Key finding:** 3-class accuracy drops to ~50%, and the model predicts zero draws across 1,100 test matches. This is not a modelling failure — it is a signal that draws are structurally unpredictable from pre-match form data. Draw rates barely move (19–26%) regardless of the form gap between teams.


In [ ]:
X_train_mn = pd.concat([train[form_features].reset_index(drop=True),
                        season_train.reset_index(drop=True)], axis=1)
X_test_mn  = pd.concat([test[form_features].reset_index(drop=True),
                         season_test.reset_index(drop=True)], axis=1)

X_train_mn_c = sm.add_constant(X_train_mn)
X_test_mn_c  = sm.add_constant(X_test_mn.reindex(columns=X_train_mn.columns, fill_value=0))

y_train_mn = train["Outcome"].reset_index(drop=True)   # 0=Away, 1=Draw, 2=Home
y_test_mn  = test["Outcome"].reset_index(drop=True)

mlogit        = MNLogit(y_train_mn, X_train_mn_c)
mlogit_result = mlogit.fit(disp=0, maxiter=1000)

pred_probs = mlogit_result.predict(X_test_mn_c)
pred_class = pred_probs.values.argmax(axis=1)
acc_mn     = accuracy_score(y_test_mn, pred_class)

print(f"3-class test accuracy: {acc_mn:.3f}")
print(f"\n── Classification Report ──")
print(classification_report(y_test_mn, pred_class, target_names=["Away Win", "Draw", "Home Win"]))

fig, ax = plt.subplots(figsize=(7, 6))
cm   = confusion_matrix(y_test_mn, pred_class)
disp = ConfusionMatrixDisplay(cm, display_labels=["Away", "Draw", "Home"])
disp.plot(ax=ax, cmap="Blues", colorbar=False)
ax.set_title("Multinomial Logit — Confusion Matrix (Test Set)\nNote: the model predicts zero draws")
plt.tight_layout()
plt.savefig("outputs/fig7_confusion_matrix.png", bbox_inches="tight")
plt.show()


---
## Part 8 — Rolling Cross-Validation

Standard k-fold CV is wrong for time-series data — it allows models to be tested on periods that precede their training data. `TimeSeriesSplit` uses expanding windows: each fold trains on everything before a cutoff and tests on the next chunk.

**Result:** 60.9% ± 1.5% across 5 folds — stable performance over time, confirming the model is not overfitting to a particular era.


In [ ]:
df_cv     = df.dropna(subset=form_features).sort_values("MatchDate").reset_index(drop=True)
s_dummies = pd.get_dummies(df_cv["SeasonStart"], prefix="S", drop_first=True)
X_cv      = pd.concat([df_cv[form_features], s_dummies], axis=1)
y_cv      = df_cv["HomeWin"]

tscv         = TimeSeriesSplit(n_splits=5)
fold_results = []

for fold, (tr_idx, te_idx) in enumerate(tscv.split(X_cv), 1):
    m = LogisticRegression(max_iter=1000)
    m.fit(X_cv.iloc[tr_idx], y_cv.iloc[tr_idx])
    acc = accuracy_score(y_cv.iloc[te_idx], m.predict(X_cv.iloc[te_idx]))
    fold_results.append({"Fold": fold, "Train": len(tr_idx), "Test": len(te_idx), "Accuracy": acc})
    print(f"  Fold {fold}: Train={len(tr_idx):,}  Test={len(te_idx):,}  Acc={acc:.3f}")

cv_df    = pd.DataFrame(fold_results)
mean_acc = cv_df["Accuracy"].mean()
std_acc  = cv_df["Accuracy"].std()
print(f"\n  Mean CV Accuracy: {mean_acc:.3f} ± {std_acc:.3f}")

fig, ax = plt.subplots(figsize=(8, 4.5))
ax.bar(cv_df["Fold"], cv_df["Accuracy"], color=COLORS["accent"], edgecolor="white", width=0.5)
ax.axhline(mean_acc, color=COLORS["covid"], linestyle="--", label=f"Mean: {mean_acc:.3f}")
ax.set_xlabel("CV Fold (Time-Ordered)")
ax.set_ylabel("Accuracy")
ax.set_title("Rolling Cross-Validation: Temporal Stability")
ax.set_ylim(0.45, 0.75)
ax.legend()
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig("outputs/fig8_rolling_cv.png", bbox_inches="tight")
plt.show()


---
## Part 9 — Deep Dives

### 9a. Fortress Rankings
Which clubs genuinely dominate at home? Minimum 100 home matches to qualify.


In [ ]:
team_home = (
    df.groupby("HomeTeam")
    .agg(
        home_matches = ("HomeWin", "count"),
        home_win_pct = ("HomeWin", "mean"),
        avg_home_gd  = ("GoalDiff", "mean"),
    )
    .sort_values("home_win_pct", ascending=False)
)
fortress = team_home[team_home["home_matches"] >= 100]

print("Top 15 Fortresses:")
print(fortress.head(15)[["home_win_pct", "avg_home_gd", "home_matches"]]
      .rename(columns={"home_win_pct": "Home Win %", "avg_home_gd": "Avg GD", "home_matches": "Matches"})
      .style.format({"Home Win %": "{:.1%}", "Avg GD": "{:+.2f}"}))


In [ ]:
top_n  = fortress.head(15).sort_values("home_win_pct", ascending=True)
colors = ["#c0392b" if t in BIG_SIX else "#3498db" for t in top_n.index]

fig, ax = plt.subplots(figsize=(11, 7))
ax.barh(range(len(top_n)), top_n["home_win_pct"].values,
        color=colors, edgecolor="white", height=0.6)
ax.set_yticks(range(len(top_n)))
ax.set_yticklabels(top_n.index, fontsize=10)
for i, (v, gd) in enumerate(zip(top_n["home_win_pct"], top_n["avg_home_gd"])):
    ax.text(v + 0.005, i, f"{v:.1%}  (GD: {gd:+.2f})", va="center", fontsize=9)
ax.set_xlabel("Home Win Rate")
ax.set_title("Premier League Fortresses (2000–2025)\nRed = Big Six  |  Min 100 home matches")
ax.xaxis.set_major_formatter(mticker.PercentFormatter(1.0))
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig("outputs/deep_fig1_fortresses.png", bbox_inches="tight")
plt.show()


### 9b. COVID Impact by Team

When the fans disappeared, whose advantage evaporated — and who actually *improved*?

In [ ]:
pre_covid = df[df["SeasonStart"] < 2019]
covid_era = df[df["SeasonStart"].isin([2019, 2020])]

pre_hw   = pre_covid.groupby("HomeTeam")["HomeWin"].agg(["mean","count"]).rename(columns={"mean":"pre_hw","count":"pre_n"})
covid_hw = covid_era.groupby("HomeTeam")["HomeWin"].agg(["mean","count"]).rename(columns={"mean":"covid_hw","count":"covid_n"})

ci = pre_hw.join(covid_hw, how="inner")
ci = ci[ci["covid_n"] >= 15]
ci["change"] = ci["covid_hw"] - ci["pre_hw"]
ci = ci.sort_values("change")

print("Biggest losers of home advantage during COVID:")
print(ci.head(8)[["pre_hw","covid_hw","change"]].style.format("{:.1%}"))

print("\nLeast affected / gainers:")
print(ci.tail(5)[["pre_hw","covid_hw","change"]].style.format("{:.1%}"))


In [ ]:
offsets = {"Man City": (15, 10), "Liverpool": (-15, -12)}
default_offset = (0, 9)

fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(ci["pre_hw"], ci["covid_hw"], s=ci["covid_n"]*4,
           alpha=0.7, color="#3498db", edgecolor="white")

for team in ci.index:
    if abs(ci.loc[team, "change"]) > 0.10 or team in BIG_SIX + ["Fulham"]:
        x, y   = ci.loc[team, "pre_hw"], ci.loc[team, "covid_hw"]
        dx, dy = offsets.get(team, default_offset)
        ax.annotate(team, (x, y), fontsize=7.5, ha="center",
                    xytext=(dx, dy), textcoords="offset points",
                    arrowprops=dict(arrowstyle="-", color="grey", lw=0.5) if team in offsets else None)

ax.plot([0.1, 0.8], [0.1, 0.8], "--", color="grey", alpha=0.5, label="No change")
ax.fill_between([0.1, 0.8], [0.1, 0.8], [0.1, 0.1], alpha=0.04, color="red")
ax.fill_between([0.1, 0.8], [0.1, 0.8], [0.8, 0.8], alpha=0.04, color="green")
ax.set_xlabel("Pre-COVID Home Win Rate (2000–2019)")
ax.set_ylabel("COVID-Era Home Win Rate (2019–2021)")
ax.set_title("Which Teams Lost Their Fortress During COVID?\nBelow the line = worse without fans")
ax.xaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.legend(fontsize=10)
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig("outputs/deep_fig2_covid_by_team.png", bbox_inches="tight")
plt.show()


### 9c. Big Six vs the Rest

The ceiling (Big Six) has held steady. The floor (everyone else) has dropped.

In [ ]:
df["HomeBigSix"] = df["HomeTeam"].isin(BIG_SIX).astype(int)

sb6   = df[df["HomeBigSix"] == 1].groupby("SeasonStart")["HomeWin"].mean()
srest = df[df["HomeBigSix"] == 0].groupby("SeasonStart")["HomeWin"].mean()

fig, ax = plt.subplots(figsize=(11, 6))
ax.plot(sb6.index,   sb6.values,   marker="o", linewidth=2, label="Big Six Home", color="#c0392b", markersize=5)
ax.plot(srest.index, srest.values, marker="s", linewidth=2, label="Rest Home",    color="#3498db", markersize=5)
ax.fill_between(sb6.index, sb6.values, srest.values, alpha=0.08, color="grey")
ax.axvspan(2019, 2020.5, alpha=0.12, color="red", label="COVID Era")
ax.set_title("Home Win Rate: Big Six vs the Rest (2000–2025)")
ax.set_ylabel("Home Win Rate")
ax.set_xlabel("Season Start Year")
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.legend(fontsize=10)
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig("outputs/deep_fig3_big_six_vs_rest.png", bbox_inches="tight")
plt.show()


### 9d. Derby & Rivalry Effects

If atmosphere still matters anywhere, it should be derbies. The data is instructive — and mixed.

In [ ]:
rivalries = {
    "North London (ARS-TOT)": ("Arsenal",    "Tottenham"),
    "Manchester (MUN-MCI)":   ("Man United", "Man City"),
    "Merseyside (LIV-EVE)":   ("Liverpool",  "Everton"),
    "NW Derby (LIV-MUN)":     ("Liverpool",  "Man United"),
    "London (CHE-ARS)":       ("Chelsea",    "Arsenal"),
    "Tyne-Wear (NEW-SUN)":    ("Newcastle",  "Sunderland"),
    "M23 Derby (BHA-CRY)":    ("Brighton",   "Crystal Palace"),
}

overall_hw   = df["HomeWin"].mean()
overall_draw = df["Draw"].mean()
rivalry_rows = []

print(f"{'Rivalry':<26} {'N':>4} {'HW%':>7} {'Draw%':>7} {'AW%':>7}")
print("-" * 56)
for name, (a, b) in rivalries.items():
    mask = (((df["HomeTeam"]==a) & (df["AwayTeam"]==b)) |
            ((df["HomeTeam"]==b) & (df["AwayTeam"]==a)))
    s = df[mask]
    if len(s) < 5:
        continue
    hw, dr, aw = s["HomeWin"].mean(), s["Draw"].mean(), s["AwayWin"].mean()
    rivalry_rows.append({"name": name, "hw": hw, "draw": dr, "aw": aw})
    print(f"{name:<26} {len(s):>4} {hw:>6.1%} {dr:>6.1%} {aw:>6.1%}")
print(f"{'LEAGUE AVERAGE':<26}      {overall_hw:>6.1%} {overall_draw:>6.1%} {df['AwayWin'].mean():>6.1%}")


In [ ]:
rdf = pd.DataFrame(rivalry_rows)
x   = range(len(rdf))
w   = 0.25

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar([i - w for i in x], rdf["hw"],   w, label="Home Win", color="#2ecc71")
ax.bar(x,                   rdf["draw"], w, label="Draw",     color="#f39c12")
ax.bar([i + w for i in x], rdf["aw"],   w, label="Away Win", color="#e74c3c")
ax.axhline(overall_hw,   color="black",   linestyle="--", alpha=0.3, label=f"League HW avg ({overall_hw:.0%})")
ax.axhline(overall_draw, color="#f39c12", linestyle="--", alpha=0.3, label=f"League Draw avg ({overall_draw:.0%})")
ax.set_xticks(x)
ax.set_xticklabels([n.replace(" (", "\n(") for n in rdf["name"]], fontsize=8, rotation=25, ha="right")
ax.set_ylabel("Proportion")
ax.set_title("Home Advantage in Major Rivalries vs League Average")
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.legend(fontsize=8.5, ncol=2)
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)
plt.tight_layout()
plt.savefig("outputs/deep_fig4_rivalries.png", bbox_inches="tight")
plt.show()


### 9e. The Draw Problem

Draw rates are stubbornly stable regardless of form differential — the model predicts zero draws. This is not a bug, it is what the data says.

In [ ]:
form_col = "Diff_Form_GD_5"
df_wf = df.dropna(subset=[form_col]).copy()
df_wf["FormBucket"] = pd.cut(
    df_wf[form_col],
    bins=[-5, -1.5, -0.5, 0.5, 1.5, 5],
    labels=["Away much\nbetter", "Away slight\nedge",
            "Even\nform", "Home slight\nedge", "Home much\nbetter"],
)

bucket_stats = df_wf.groupby("FormBucket", observed=True).agg(
    hw=("HomeWin","mean"), dr=("Draw","mean"), aw=("AwayWin","mean"), n=("Draw","count")
).reset_index()

draw_matches = df[df["Draw"] == 1]
scores       = draw_matches.groupby("FullTimeHomeGoals").size()

fig, axes = plt.subplots(1, 2, figsize=(14, 5.5))

ax = axes[0]
x = range(len(bucket_stats))
ax.bar([i - w for i in x], bucket_stats["hw"], w, label="Home Win", color="#2ecc71")
ax.bar(x,                   bucket_stats["dr"], w, label="Draw",     color="#f39c12")
ax.bar([i + w for i in x], bucket_stats["aw"], w, label="Away Win", color="#e74c3c")
ax.set_xticks(x)
ax.set_xticklabels(bucket_stats["FormBucket"], fontsize=9)
ax.set_ylabel("Proportion")
ax.set_title("Panel A: Outcomes by Form Gap")
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.legend(fontsize=9)
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

ax = axes[1]
bars = ax.bar(scores.index.astype(int), scores.values / len(draw_matches),
              color="#f39c12", edgecolor="white", width=0.6)
for g, v in zip(scores.index, scores.values / len(draw_matches)):
    ax.text(g, v + 0.01, f"{v:.0%}", ha="center", fontsize=10, fontweight="bold")
ax.set_xlabel("Goals per side")
ax.set_ylabel("Proportion of draws")
ax.set_title("Panel B: Score Distribution of Draws")
ax.set_xticks(range(6))
ax.set_xticklabels(["0-0", "1-1", "2-2", "3-3", "4-4", "5-5"])
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
for s in ["top", "right"]:
    ax.spines[s].set_visible(False)

plt.suptitle("The Draw Problem: Football's Stubbornly Unpredictable Middle Ground",
             fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.savefig("outputs/deep_fig5_draws.png", bbox_inches="tight")
plt.show()


---
## Conclusions

| Finding | Detail |
|---------|--------|
| **Home advantage is declining** | ~1pp every 3–4 years since 2000; the LOWESS trend is persistent |
| **COVID was an accelerant, not the cause** | Home win rate dropped 46.5% → 41.6% (p=0.009), recovered only to 44.7% |
| **Structural vs atmospheric advantage** | Man City/Liverpool *improved* during COVID; Fulham collapsed from 44% → 10.5% |
| **Form predicts direction, not draws** | 61% binary accuracy; 0 draws predicted across 1,100 test matches |
| **Team FE adds almost nothing** | +0.1pp — form differentials are not proxies for squad quality |
| **CV stability** | 60.9% ± 1.5% across 5 time-ordered folds |

> *The 12th man is not dead. But for a growing number of clubs, he is retired.*

---

**Read the full blog post:** [The Death of the 12th Man](https://ammartyabji.wixsite.com/mysite/post/the-death-of-the-12th-man-how-home-advantage-is-fading-in-the-premier-league)  
**Author:** Ammar Tyabji · [GitHub @AmmardoesStats](https://github.com/AmmardoesStats)
